In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1,784)/255
x_test = x_test.reshape(-1,784)/255
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(784, activation='relu'),  # Input layer
    tf.keras.layers.Dense(256, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer
])
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=50)
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/10
1200/1200 [==============================] - 13s 10ms/step - loss: 0.1879 - accuracy: 0.9422
Epoch 2/10
1200/1200 [==============================] - 11s 9ms/step - loss: 0.0766 - accuracy: 0.9762
Epoch 3/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0513 - accuracy: 0.9839
Epoch 4/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0382 - accuracy: 0.9874
Epoch 5/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0316 - accuracy: 0.9894
Epoch 6/10
1200/1200 [==============================] - 14s 11ms/step - loss: 0.0245 - accuracy: 0.9920
Epoch 7/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0223 - accuracy: 0.9929
Epoch 8/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0203 - accuracy: 0.9930
Epoch 9/10
1200/1200 [==============================] - 14s 11ms/step - loss: 0.0160 - accuracy: 0.9947
Epoch 10/10
313/313 [==============================] - 1s 2ms/ste

In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape input data to 4D (samples, height, width, channels)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1376 - accuracy: 0.9596 - val_loss: 0.0478 - val_accuracy: 0.9851
Epoch 2/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0463 - accuracy: 0.9851 - val_loss: 0.0342 - val_accuracy: 0.9887
Epoch 3/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0320 - accuracy: 0.9900 - val_loss: 0.0352 - val_accuracy: 0.9882
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0242 - accuracy: 0.9921 - val_loss: 0.0297 - val_accuracy: 0.9906
Epoch 5/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0184 - accuracy: 0.9941 - val_loss: 0.0344 - val_accuracy: 0.9899
Epoch 6/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0150 - accuracy: 0.9952 - val_loss: 0.0287 - val_accuracy: 0.9914
Epoch 7/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0115 - accuracy: 0.9961 - val_loss: 0.0324 -

In [ ]:
#Hyperparameter Optimisation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Add a channel dimension for grayscale images
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    
    # Tune the number of filters and kernel size in the convolutional layers
    hp_filters = hp.Int('filters', min_value=32, max_value=128, step=32)
    hp_kernel_size = hp.Choice('kernel_size', values=[3, 5])
    
    model.add(layers.Conv2D(filters=hp_filters, kernel_size=hp_kernel_size, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(filters=hp_filters, kernel_size=hp_kernel_size, activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Flatten())
    
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the dense layer
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(layers.Dropout(rate=hp_dropout))
    
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_directory',
    project_name='mnist_tuning'
)

# Perform the hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get the best model and print the summary
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Trial 1 Complete [00h 09m 46s]
val_accuracy: 0.9930999875068665

Best val_accuracy So Far: 0.9930999875068665
Total elapsed time: 00h 09m 46s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
32                |32                |filters
3                 |3                 |kernel_size
416               |96                |units
0.2               |0.4               |dropout

Epoch 1/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1360 - accuracy: 0.9579 - val_loss: 0.0457 - val_accuracy: 0.9852
Epoch 2/10
 802/1875 [===========>..................] - ETA: 15s - loss: 0.0450 - accuracy: 0.9860